# GLOG: calculate lambda



---

## 1. Import Packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.optimize
import qcrsc   

print('All packages successfully loaded')

# Remove later
%load_ext autoreload
%autoreload 2

## 2. Load Data

In [ ]:
home = 'data/'
file = 'MTBLS290.xlsx' 

DataTable, PeakTable = qcrsc.load_dataXL(home + file,'Data','Peak')

## 3. Extract X

In [ ]:
peaklist = PeakTable.Name
x = DataTable.loc[:, peaklist]
xqc = x[DataTable.SampleType == 'QC']

xqc = np.array(xqc)

## 4. Plot variance vs. mean

In [ ]:
var = np.var(xqc, axis=0)
mean = np.mean(xqc, axis=0)

# Put into pd df for seaborn
d = {'var':var, 'mean':mean}
df = pd.DataFrame(d)

sns.scatterplot(x="mean", y="var", data=df) # 1534 peaks

## 5. Optimisation of Lambda

Based on Matlab code from the following publication:
https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-8-234#MOESM2

Improved classification accuracy in 1- and 2-dimensional NMR metabolomics data using the variance stabilising generalised logarithm transformation

Helen M Parsons, Christian Ludwig, Ulrich L Günther & Mark R Viant


In [ ]:
## FUNCTIONS ##

def glog(x, lambda0):
    "glog transformation"
    y = np.log(x + np.sqrt(x ** 2 + lambda0))
    return y

def jglog(x, lambda0):
    """
    Rescale variables using Jacobian: w = J*z
    Note slight difference in format to Durbin paper - makes eqn computational
    (has extra multiplicative term only; moving minimum up)
    """
    z = glog(x, lambda0)
    gmn = np.exp(np.mean(np.log(np.sqrt(x ** 2 + lambda0)), axis=1))
    zj = np.array(z.T * gmn).T
    return zj, gmn

def SSE(lambda0, x):
    "Calculate SSE"
    L = len(x.T) # Num of mets (QCs)
    N = len(x) # Num of samples (QCs)

    z = jglog(x, lambda0)[0]
    grand_mean = np.mean(z)
    mean_spec = np.mean(z, axis=0)

    s = 0
    for i in range(N):
        row_mean = np.mean(z[i,:]) # Mean per sample
        for j in range(L):
            col_mean = np.mean(z[:,j]) # Mean per metabolite
            s = s + (z[i,j] - mean_spec[j]) ** 2
    return s


In [ ]:
## Determine optimal lambda ##


# Replace nans with metabolite means ?
# xqc = np.where(np.isnan(xqc), np.ma.array(xqc, mask = np.isnan(xqc)).mean(axis = 0), xqc)    

# Find starting point (Durbin & Rocke, 2003)
Lm = np.median(xqc_nonan, axis=0)
Lm = np.median(Lm)
lambda0 = Lm**2

# Options: uses fminsearch
TolX = 1e-16
TolFun = 1e-15
MaxFunEvals =  1e3
MaxIter = 1e3

# Build model
opt_lambda = scipy.optimize.fmin(SSE,
                                 lambda0,
                                 args=(xqc,),
                                 xtol=TolX, 
                                 ftol=TolFun,
                                 maxiter=MaxIter,
                                 maxfun=MaxFunEvals)
opt_lambda = opt_lambda[0]

print("Optimal lambda is: {}".format(opt_lambda))


In [ ]:
## Use optimal lambda

x_glog = glog(x, opt_lambda)